In [171]:
from openai import OpenAI

key = input("API key")
client = OpenAI(api_key=key)

In [172]:
filename = "world_df.csv"
assistant_name = "data-analyst-v0.1"

In [173]:
# Check if file is already uploaded
filelist = client.files.list(purpose="assistants")
filenames =  [x.filename for x in filelist.data]

# Upload a file with an "assistants" purpose or use existing one
if not filename in filenames:
  file = client.files.create(
    file=open(filename, "rb"),
    purpose='assistants'
  )
else:
  for f in filelist:
    if f.filename == filename:
      file = client.files.retrieve(f.id)
      break

In [174]:


# Check if assistant already exists
assistant_list = client.beta.assistants.list()
assistant_names =  [x.name for x in assistant_list.data]
assistant_exists = True if assistant_name in assistant_names else False

if not assistant_exists:
  # Create an assistant using the file ID
  assistant = client.beta.assistants.create(
    name = "data-analyst-v0.1",
    instructions="You are a data analyst",
    model="gpt-4o",
    tools=[{"type": "code_interpreter"}],
    tool_resources={
      "code_interpreter": {
        "file_ids": [file.id]
      }
    }
  )
else:
    for a in assistant_list:
      if a.name == assistant_name:
        assistant = client.beta.assistants.retrieve(a.id)
        break



In [175]:
prompt = "Using the csv file attached, display a graph of 'Year' against 'Anuual CO2 emissions"
prompt = "Using the csv file attached, display a graph of 'Year' against all other columns"
prompt = "Using the csv file attached, display a graph of 'Year' against all other columns. Only chart the figures for the 21st century"

In [176]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": prompt,
      "attachments": [
        {
          "file_id": file.id,
          "tools": [{"type": "code_interpreter"}]
        }
      ]
    }
  ]
)

In [177]:
image_file_name = "my-image.png"
image_file_name = "my-image2.png"
image_file_name = "my-image3.png"


In [178]:
from typing_extensions import override
from openai import AssistantEventHandler

# First, we create a EventHandler class to define
# how we want to handle the events in the response stream.

class EventHandler(AssistantEventHandler):
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > {text.value}", end="", flush=True)

  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)

  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)

  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)

# Then, we use the `stream` SDK helper
# with the `EventHandler` class to create the Run
# and stream the response.

with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions=f"create a downloadable file for the graph called {image_file_name}",
  event_handler=EventHandler(),
) as stream:
  stream.until_done()


assistant > code_interpreter

import pandas as pd

# Load the CSV file
file_path = '/mnt/data/file-8XwqMOlaH6hoKEEKOYXPYqTh'
data = pd.read_csv(file_path)

# Display the first few rows of the dataframe to understand its structure
data.head()import matplotlib.pyplot as plt

# Filter the data to include only the 21st century (from year 2000 onwards)
data_21st_century = data[data['Year'] >= 2000]

# Define the columns to be plotted against 'Year'
columns_to_plot = data.columns.drop(['Entity', 'Code', 'Year'])

# Plot the data
plt.figure(figsize=(12, 8))
for column in columns_to_plot:
    plt.plot(data_21st_century['Year'], data_21st_century[column], label=column)

plt.title('Yearly Data in the 21st Century')
plt.xlabel('Year')
plt.ylabel('Values')
plt.legend(title='Metrics', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()

# Save the plot to a file
plot_file_path = '/mnt/data/my-image3.png'
plt.savefig(plot_file_path)

plt.show()

plot_file_path

output >

a

In [179]:
"""
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="create a downloadable file for the graph."
)

"""

'\nrun = client.beta.threads.runs.create_and_poll(\n  thread_id=thread.id,\n  assistant_id=assistant.id,\n  instructions="create a downloadable file for the graph."\n)\n\n'

In [180]:
"""
if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)
"""

"\nif run.status == 'completed': \n  messages = client.beta.threads.messages.list(\n    thread_id=thread.id\n  )\n  print(messages)\nelse:\n  print(run.status)\n"

In [181]:
print(client.beta.threads.messages.list(thread.id).data)

[Message(id='msg_DK9074gkvN0koiBZ2Fh8pCJy', assistant_id='asst_8phupECTtHyRHrPzTlQHScQZ', attachments=[Attachment(file_id='file-Ad6E6gBQxoOgg8ucQEA5LHXA', tools=[CodeInterpreterTool(type='code_interpreter')])], completed_at=None, content=[ImageFileContentBlock(image_file=ImageFile(file_id='file-ie5gJxvDBtCUEKWrZp0NK4eY', detail=None), type='image_file'), TextContentBlock(text=Text(annotations=[FilePathAnnotation(end_index=152, file_path=FilePath(file_id='file-Ad6E6gBQxoOgg8ucQEA5LHXA'), start_index=121, text='sandbox:/mnt/data/my-image3.png', type='file_path')], value='The graph has been created and saved successfully. You can download the file using the link below:\n\n[Download the graph](sandbox:/mnt/data/my-image3.png)'), type='text')], created_at=1717856875, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_2hH85ttOGf1VLOeAu3BMlDta', status=None, thread_id='thread_UitEqs6R1OBp5E32vEaNY0EQ'), Message(id='msg_QyRFg30WEoAx

In [182]:
filelist = client.files.list(purpose="assistants_output")

In [183]:
image_list = [x for x in filelist.data if image_file_name in x.filename]

In [184]:
image_list[0]

FileObject(id='file-Ad6E6gBQxoOgg8ucQEA5LHXA', bytes=149936, created_at=1717856876, filename='/mnt/data/my-image3.png', object='file', purpose='assistants_output', status='processed', status_details=None)

In [185]:
id = image_list[0].id # the last in the list is the latest

In [186]:
id
#file-mXxnpTb8tbE7Jv1tPErOd85v


'file-Ad6E6gBQxoOgg8ucQEA5LHXA'

In [187]:
client.files.retrieve(id)

FileObject(id='file-Ad6E6gBQxoOgg8ucQEA5LHXA', bytes=149936, created_at=1717856876, filename='/mnt/data/my-image3.png', object='file', purpose='assistants_output', status='processed', status_details=None)

In [188]:
client.files.content(id)

In [189]:
image_data = client.files.content(id)
image_data_bytes = image_data.read()

with open(f"./{image_file_name}", "wb") as file:
    file.write(image_data_bytes)


![](my-image2.png)

![](my-image3.png)